In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date, datetime, timedelta
import time
import yfinance as yf
from pyetfdb_scraper import etf
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
import zipfile
from io import BytesIO

import relative_value as rv

from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import vix_utils as vix

import asyncio 
import nest_asyncio
nest_asyncio.apply()

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import selenium.common.exceptions


chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.page_load_strategy = 'eager'


from statsmodels.graphics.tsaplots import plot_acf


from sklearn.linear_model import LinearRegression

In [16]:

df = yf.Ticker('UVXY').history(start=datetime(2022,1,1), end=datetime(2023,7,6), interval='60m')

df = df.drop(['High', 'Low','Dividends','Stock Splits','Capital Gains'],axis=1)
df['return_close'] = df['Close'].pct_change()
df['return_close_lag'] = df['return_close'].shift(1)
#px.scatter(df, x='return_close_lag', y='return_close')
df[['return_close_lag', 'return_close']].corr()
#px.line(df, x=df.index, y = 'return_close')
df['return_close_2'] = df['Close'].pct_change(2)
df['return_close_3'] = df['Close'].pct_change(3)
df['return_close_4'] = df['Close'].pct_change(4)
#df[['return_close', 'return_close_2', 'return_close_3', 'return_close_4']].corr()
#px.histogram(df[df['return_close_lag'] > 0.05],x='return_close')
#df[df['return_close_lag'] < 0.1]['return_close'].mean() 

df.index = df.index.tz_localize(None)

In [3]:
vix_contango = rv.get_historical_vix_contango(datetime(2020,1,1), datetime(2023,6,30))

/home/wiltonwking/IV Analysis/relative_value.py:104: GuessedAtParserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 104 of the file /home/wiltonwking/IV Analysis/relative_value.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.




In [19]:
df = df.join(vix_contango['1-2'], how='inner')

In [20]:
df = df.rename(columns={'1-2': 'vix_contango'})

In [21]:
df['vix_contango'] = pd.to_numeric(df['vix_contango'].str.replace('%',''))

In [22]:
px.line(df, x=df.index, y='Close')

In [23]:
px.line(df, x=df.index, y='vix_contango')

In [25]:
df['day_of_month'] = df.index.day

In [26]:
px.box(df,x='day_of_month',y='return_close')

In [28]:
df['vix_contango_lag'] = df['vix_contango'].shift(1)

In [29]:
df[['vix_contango_lag','return_close']].corr()

,vix_contango_lag,return_close
vix_contango_lag,1.00000,-0.04286
return_close,-0.04286,1.00000


In [30]:
px.scatter(df,x='vix_contango_lag',y='return_close')

In [11]:
econ_df = pd.read_csv('calendar-event-list.csv', index_col=1)

In [10]:
econ_df.index = pd.to_datetime(econ_df.index).date

In [17]:
econ_df = econ_df[econ_df['Impact'].isin(['MEDIUM','HIGH'])]

In [18]:
vix_hist = yf.Ticker('^VIX').history(period='max')